# Now that you’ve found the answers to the questions above, design two of your own questions to answer. 

For the first question, We're going to be looking at the average time each comment was posted, to see which subreddits post the most during the day, and which have the most night owls.

In [54]:
import re
import pandas as pd
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, col, desc, explode, lower, mean, from_unixtime, split, count
from pyspark.sql.types import StructType, StructField, FloatType, LongType, StringType, BooleanType, ArrayType, IntegerType

sqlContext = SQLContext(sc)

df = sqlContext.read.json("hdfs://orion11:15001/sampled_reddit/*")
columns = [
    "distinguished",
    "downs",
    "created_utc",
    "controversiality",
    "edited",
    "gilded",
    "author_flair_css_class",
    "id",
    "author",
    "retrieved_on",
    "score_hidden",
    "subreddit_id",
    "score",
    "name",
    "author_flair_text",
    "link_id",
    "archived",
    "ups",
    "parent_id",
    "subreddit",
    "body"]

df = df.select("created_utc", "subreddit")


In [55]:
# convert timestamp to string
    # note: According to the docs, it should convert it to my local timezone
time_df = df.withColumn('date_time', from_unixtime('created_utc', format="HH:mm:ss"))

split_col = split(time_df['date_time'], ':')
time_df = time_df.withColumn('hour', split_col.getItem(0).cast(IntegerType()))
time_df = time_df.withColumn('min', split_col.getItem(1).cast(IntegerType()))
time_df = time_df.withColumn('sec', split_col.getItem(2).cast(IntegerType()))
time_df = time_df.drop("created_utc", "date_time")
time_df.show()

+-----------------+----+---+---+
|        subreddit|hour|min|sec|
+-----------------+----+---+---+
|        AskReddit|  17|  0| 10|
|        AskReddit|  17|  0| 12|
|         politics|  17|  0| 16|
|        AskReddit|  17|  0| 19|
|             IAmA|  17|  0| 25|
|       California|  17|  0| 28|
|             pics|  17|  0| 32|
|       reddit.com|  17|  0| 38|
|        AskReddit|  17|  0| 34|
|        AskReddit|  17|  0| 41|
|           Frugal|  17|  0| 45|
|           trance|  17|  0| 48|
|           google|  17|  0| 48|
|             IAmA|  17|  0| 55|
|         sandiego|  17|  0| 57|
|       googleplus|  17|  1|  4|
|       technology|  17|  1|  8|
|        worldnews|  17|  1|  5|
|DrawingFromWithin|  17|  1|  1|
|           gaming|  17|  1|  7|
+-----------------+----+---+---+
only showing top 20 rows



In [62]:
# now group by subreddit, and find the averages
grouped_df = time_df.groupBy('subreddit').agg(mean('hour').cast(IntegerType()).alias("avg_hour"), 
                                              mean('min').cast(IntegerType()).alias("avg_min"), 
                                              mean('sec').cast(IntegerType()).alias("avg_sec"), 
                                              count('subreddit').alias('num_posts'))

# ignore small subreddits (50 fo sample_v3, 1000 for larger sample)
grouped_df = grouped_df.filter("num_posts >= 1000")

In [63]:
# Display all the day people
grouped_df.orderBy(["avg_hour", "avg_min", "avg_sec"], ascending=[False, False, False]).show(25)

+--------------------+--------+-------+-------+---------+
|           subreddit|avg_hour|avg_min|avg_sec|num_posts|
+--------------------+--------+-------+-------+---------+
|             alienth|      20|     13|     29|     1086|
|              Oscars|      18|     29|     30|     1798|
|             Toonami|      18|     29|     29|    45023|
|      azdiamondbacks|      17|     29|     29|    12145|
|          LAClippers|      17|     29|     29|    26708|
|   secretsubgonewild|      17|     29|     28|     1101|
|     LiveFromNewYork|      16|     30|     29|    20151|
|                 agt|      16|     29|     29|     2386|
|tamrielscholarsguild|      16|     29|     29|     5366|
|            SFGiants|      16|     29|     29|   121490|
|             Dodgers|      16|     29|     29|   124437|
|     ColoradoRockies|      16|     29|     29|    12204|
|            Mariners|      16|     29|     29|    73122|
|       SanJoseSharks|      16|     29|     29|    30490|
|       Gekkou

In [64]:
# Display all the night owls
grouped_df.orderBy(["avg_hour", "avg_min", "avg_sec"]).show(25)

+-----------------+--------+-------+-------+---------+
|        subreddit|avg_hour|avg_min|avg_sec|num_posts|
+-----------------+--------+-------+-------+---------+
|      nameaserver|       4|     25|     29|     5975|
|     newonspotify|       4|     29|     29|     1085|
|         BayStars|       6|     28|     29|     1666|
|Suicidal_Insanity|       6|     29|     29|     1048|
| VOCALOID_UTAU_jp|       6|     30|     29|     2465|
|           hampan|       7|     29|     29|     7070|
|            italy|       7|     29|     29|   119738|
|            TV_ja|       7|     29|     29|     2088|
|           france|       7|     29|     29|   188867|
|         CasualUK|       7|     29|     29|     4912|
|      Southampton|       7|     29|     29|     2405|
|   F1FeederSeries|       7|     29|     29|     2733|
|          belgium|       7|     29|     29|    43207|
|             nanJ|       7|     30|     29|     1763|
|       FreeEBOOKS|       8|     22|     29|     3253|
|      gun

## Analysis

When I started this analysis, I think I went into it with a very naieve expectation. I though in the early morning I would see subreddits about insomnia, or about horror movies. Instead, I found an active community off redditors from other countries, which I had not even thought about when originally coming up with this question. For example, when diplaying the "day people", I saw subreddits for sports teams, starting with teams on the west coast (LA clippers, or The Warriors), moving towards east coast teams and cities as the time changed. Looking at what I thought would be "night owls", we saw subreddits for EU countries like France or Italy, along with some FC (soccer) subreddits.

Not to be vulgar, but there was also not as much porn as I would have expected, though this may be because of my filter removing smaller subreddits.